In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pprint
import random

import yaml
from pydash import py_

In [8]:
path_to_settings = Path(".") / "settings.yml"
assert path_to_settings.exists(), f"Settings file not found at {path_to_settings}"

with open(path_to_settings, "r") as f:
    settings = yaml.safe_load(f)
pprint(settings)

{'attendee_group1': ['Helena Urquijo',
                     'Andrew Elmore',
                     'Yueying Li',
                     'Josine Min',
                     'Haotian Tang',
                     'Matt Boyton',
                     'Zhaozhen Xu',
                     'Si Fang',
                     'Xuyutian Wang',
                     'Winfred Gatua',
                     'Phoebe Dickson',
                     'Amanda Chong',
                     'Andrea Santangelo',
                     'Olalekan Awoniran',
                     'Yi Liu'],
 'attendee_group2': ['Lavinia Paternoster',
                     'Louise Millard',
                     'Venexia Walker',
                     'Tom Gaunt'],
 'params': {'group_size': 4, 'num_groups': 5, 'seed': 42},
 'projects': ['a', 'b', 'c', 'd']}


In [5]:
random.seed(settings["params"]["seed"])
raw_list = [1, 2, 3, 4]
py_.shuffle(raw_list)

[3, 2, 4, 1]

In [ ]:
# determin group size from number of groups and length of attendees
attendees = settings["attendee_group1"]
num_groups = settings["params"]["num_groups"]
group_size = len(attendees) // num_groups
print(group_size)

3


In [17]:
group1_raw = settings["attendee_group1"]
group1_size = len(group1_raw) // settings["params"]["num_groups"]
group1_chunked = py_.chain(group1_raw) \
    .shuffle().chunk(group1_size) \
        .value()
print(len(group1_chunked))

5


In [20]:
py_.chunk([1, 2, 3, 4, 5], 2)

[[1, 2], [3, 4], [5]]

In [33]:
def allocate_people_to_groups(people, num_groups):
    # Shuffle the list of people to ensure random distribution
    random.shuffle(people)
    
    # Create empty groups
    groups = [[] for _ in range(num_groups)]
    
    # Distribute people into groups
    for i, person in enumerate(people):
        groups[i % num_groups].append(person)
    
    return groups

people = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Hannah"]
num_groups = 3
allocate_people_to_groups(people, num_groups)

[['David', 'Charlie', 'Alice'], ['Frank', 'Hannah', 'Eve'], ['Grace', 'Bob']]

In [41]:
groups2_raw = settings["attendee_group2"]
group2 = allocate_people_to_groups(groups2_raw, settings["params"]["num_groups"])

In [45]:
projects = py_.shuffle(settings["projects"])[:group_size]

In [46]:
alloc = []
for _ in zip(group1_chunked, group2, projects):
    team = py_.flatten(_[:2])
    proj = _[2]
    item = {
        "team": team,
        "proj": proj,
    }
    alloc.append(item)
alloc

[{'team': ['Phoebe Dickson',
   'Josine Min',
   'Amanda Chong',
   'Louise Millard',
   'Tom Gaunt',
   'Venexia Walker'],
  'proj': 'a'},
 {'team': ['Helena Urquijo',
   'Andrea Santangelo',
   'Yi Liu',
   'Lavinia Paternoster',
   'Venexia Walker',
   'Lavinia Paternoster'],
  'proj': 'd'},
 {'team': ['Xuyutian Wang',
   'Winfred Gatua',
   'Yueying Li',
   'Louise Millard',
   'Tom Gaunt'],
  'proj': 'c'}]